### Clean BEP Companies

_Cleans BEP companies' certifications and cleans and geocodes their physical addresses. Outputs three CSV files: `bep_company_addresses_all.csv`, `bep_company_addresses_physical_geocoded.csv`, and `bep_company_certifications`. NOTE: As a next step, more extensive record linkage should be performed (i.e., consolidating entries that have the same street address and similar names or owners.) We could also get better reseults by normalizing/standardizing the addresses before the forward geocoding step._

#### Setup

In [1]:
# Python standard library
import json
import os
import pandas as pd
import time

# HTTP requests
import requests

# Environment variable management
import dotenv

In [2]:
# Load environment variables into current environment
dotenv.load_dotenv()

True

#### Load BEP-Certified Vendors

In [3]:
# Read in BEP certifications dataset into DataFrame
vendor_dir = "../data/raw/vendors"
bep_fname = "illinois_bep_vendor_certifications_as_of_20230614.csv"
bep_df = pd.read_csv(f"{vendor_dir}/{bep_fname}", encoding='cp1252')
bep_df.head(2)

,Company Name,DBA Name,Owner First,Owner Last,Physical Address,City,State,Zip,Mailing Address,City.1,...,Certification Type,Ethnicity,Gender,Renewal,Expiration,Capability,County,Region,Reciprocal Certification Agency,Commodity Codes
0,#1 Professional Cleaning Services,NaN,Howard,Glasper,2205 Mayfair,Westchester,IL,60154.0,P.O. Box 7555,Westchester,...,MBE,African American,Male,8/17/2023,8/17/2029,91039\tJanitorial and Custodial Services,Cook,Metro Chicago,NaN,91039 - Janitorial and Custodial Services
1,"#9 Design, LLC",North Arrow Architecture,Dulcinea,Gillman,524 West Saint Charles Road,Villa Park,IL,60181.0,28 East Saint Charles Road,Villa Park,...,WBE,Caucasian,Female,12/15/2023,12/15/2023,"NIGP\t90714\tArchitectural Services, Non-Licen...",DuPage,Northern Illinois,WBDC,"90714 - Architectural Services, Non-Licensed (..."


#### Clean BEP Certifications

In [4]:
# Drop records missing a certification type
# NOTE: The system generating the data, "B2GNow", leaves the last
# few rows in the CSV file empty before including a line of metadata:
# "Generated from the B2Gnow System".
print("There are", len(bep_df), "vendor certification rows in the DataFrame.")
idx_to_drop = bep_df.query("`Certification Type` != `Certification Type`").index
bep_df = bep_df.drop(index=idx_to_drop)
print("After dropping blank records,", len(bep_df), "rows remain.")

There are 7328 vendor certification rows in the DataFrame.
After dropping blank records, 7325 rows remain.


In [5]:
# Create new column for agency certification
bep_df['Certification'] = bep_df['Agency'] + ' ' + bep_df['Certification Type']

In [6]:
# Merge on company name and create new certification columns
bep_certs_df = bep_df.loc[:, ['Company Name', 'Certification', 'Renewal', 'Expiration']]
bep_certs_df.head(10)

,Company Name,Certification,Renewal,Expiration
0,#1 Professional Cleaning Services,CEI BEP MBE,8/17/2023,8/17/2029
1,"#9 Design, LLC",CEI BEP WBE,12/15/2023,12/15/2023
2,'Bouche' Enterprises LLC,Chicago MBE,4/1/2024,4/1/2024
3,'Bouche' Enterprises LLC,Chicago WBE,4/1/2024,4/1/2024
4,'Bouche' Enterprises LLC,CEI BEP WMBE,4/1/2024,4/1/2024
5,'Merica Made Painting & Decorating Corp.,Chicago MBE,9/1/2023,9/1/2023
6,*VDR Development Group,CEI BEP MBE,1/18/2024,1/18/2024
7,@Contract Partners LLC,Chicago WBE,6/15/2023,5/15/2026
8,@Contract Partners LLC,CEI BEP WBE,7/15/2023,7/15/2023
9,"@Veterans Construction Group, LLC.",CEI BEP MBE,9/14/2023,9/14/2028


In [7]:
# Write certifications to file
fpath = "../data/clean/bep_company_certifications.csv"
bep_certs_df.to_csv(fpath, index=False)

#### Clean BEP Company Addresses

In [8]:
def build_address_dataset(
    bep_df: pd.DataFrame,
    company_name_col: str,
    street_addr_col: str,
    city_col: str,
    state_col: str,
    zip_col: str,
    address_type: str) -> pd.DataFrame:
    """
    Builds an address dataset.
    """
    # Take subset of BEP DataFrame
    str_addr_cols = [street_addr_col, city_col]
    addr_cols = str_addr_cols + [state_col, zip_col]
    subset_cols = [company_name_col] + addr_cols
    subset = bep_df.copy()[subset_cols]

    # Standardize capitalization
    def correct_case(address: str):
        corrected = []
        for word in address.split():
            if word[0].isdigit():
                corrected.append(word)
            else:
                corrected.append(word.title())
        return ' '.join(corrected)
    subset[str_addr_cols] = subset[str_addr_cols].apply(lambda s: s.apply(correct_case))

    # Standardize data type of zip code columns
    subset[zip_col] = subset[zip_col].astype(int)

    # Add new column for address type
    subset["Address Type"] = address_type

    # Define local function to dedupe addresses
    def dedupe(df):
        if len(df) == 1:
            return df
        is_po = lambda s: s.lower().strip().startswith("po")
        df = df[~df[street_addr_col].apply(is_po)]
        return df.head(1)
    
    # Define final column mapping
    col_map = {
        company_name_col: "Company Name",
        "Address Type": "Address Type",
        street_addr_col: "Street",
        city_col: "City",
        state_col: "State",
        zip_col: "ZIP Code"
    }
    
    # Reshape DataFrame
    addresses_df = (subset
        .groupby(company_name_col)
        .apply(dedupe)
        .drop(columns=company_name_col)
        .reset_index()
        .drop(columns="level_1")
        .rename(columns=col_map)
        .loc[:, list(col_map.values())])
    
    return addresses_df

In [9]:
# Correct physical addresses
physical_address_df = build_address_dataset(
    bep_df,
    company_name_col="Company Name",
    street_addr_col="Physical Address",
    city_col="City",
    state_col="State",
    zip_col="Zip",
    address_type="Physical"
)

# Preview DataFrame
physical_address_df.head(10)

,Company Name,Address Type,Street,City,State,ZIP Code
0,#1 Professional Cleaning Services,Physical,2205 Mayfair,Westchester,IL,60154
1,"#9 Design, LLC",Physical,524 West Saint Charles Road,Villa Park,IL,60181
2,'Bouche' Enterprises LLC,Physical,3511 North Osceola Avenue,Chicago,IL,60634
3,'Merica Made Painting & Decorating Corp.,Physical,6221 S Austin Ave,Chicago,IL,60638
4,*VDR Development Group,Physical,168 N Brandon Drive,Glendale Heights,IL,60190
5,"1052 Partners, Inc. dba PeopleCaddie",Physical,"5600 N River Rd, Suite 800",Rosemont,IL,60018
6,123 Labor Inc.,Physical,8318 Rutherford Ave,Burbank,IL,60459
7,1822 Enterprises LLC,Physical,"4524 S King Dr, Unit 3, Unit 3",Chicago,IL,60653
8,"1st Aerow Valuation Group, LLC",Physical,2706 E. 76th Pl.,Chicago,IL,60649
9,"1st Call Technical Services, Inc",Physical,214 Butternut Drive,Bolingbrook,IL,60440


In [10]:
# Correct mailing addresses
mailing_address_df = build_address_dataset(
    bep_df,
    company_name_col="Company Name",
    street_addr_col="Mailing Address",
    city_col="City.1",
    state_col="State.1",
    zip_col="Zip.1",
    address_type="Mailing"
)

# Preview DataFrame
mailing_address_df.head(10)

,Company Name,Address Type,Street,City,State,ZIP Code
0,#1 Professional Cleaning Services,Mailing,P.O. Box 7555,Westchester,IL,60154
1,"#9 Design, LLC",Mailing,28 East Saint Charles Road,Villa Park,IL,60181
2,'Bouche' Enterprises LLC,Mailing,3511 North Osceola Avenue,Chicago,IL,60634
3,'Merica Made Painting & Decorating Corp.,Mailing,6221 S Austin Ave,Chicago,IL,60638
4,*VDR Development Group,Mailing,168 N Brandon Drive,Glendale Heights,IL,60190
5,"1052 Partners, Inc. dba PeopleCaddie",Mailing,"5600 N River Rd, Suite 800",Rosemont,IL,60018
6,123 Labor Inc.,Mailing,8318 Rutherford Ave,Burbank,IL,60459
7,1822 Enterprises LLC,Mailing,"4524 S King Dr, Unit 3, Unit 3",Chicago,IL,60653
8,"1st Aerow Valuation Group, LLC",Mailing,2706 E. 76th Pl.,Chicago,IL,60649
9,"1st Call Technical Services, Inc",Mailing,P.O. Box 258,Bolingbrook,IL,60440


In [11]:
# Combine address datasets
address_df = (pd
    .concat([physical_address_df, mailing_address_df])
    .sort_values(by=['Company Name', 'Address Type']))

# Preview DataFrame
address_df.head(10)

,Company Name,Address Type,Street,City,State,ZIP Code
0,#1 Professional Cleaning Services,Mailing,P.O. Box 7555,Westchester,IL,60154
0,#1 Professional Cleaning Services,Physical,2205 Mayfair,Westchester,IL,60154
1,"#9 Design, LLC",Mailing,28 East Saint Charles Road,Villa Park,IL,60181
1,"#9 Design, LLC",Physical,524 West Saint Charles Road,Villa Park,IL,60181
2,'Bouche' Enterprises LLC,Mailing,3511 North Osceola Avenue,Chicago,IL,60634
2,'Bouche' Enterprises LLC,Physical,3511 North Osceola Avenue,Chicago,IL,60634
3,'Merica Made Painting & Decorating Corp.,Mailing,6221 S Austin Ave,Chicago,IL,60638
3,'Merica Made Painting & Decorating Corp.,Physical,6221 S Austin Ave,Chicago,IL,60638
4,*VDR Development Group,Mailing,168 N Brandon Drive,Glendale Heights,IL,60190
4,*VDR Development Group,Physical,168 N Brandon Drive,Glendale Heights,IL,60190


In [12]:
# Write datasets to file
address_df.to_csv("../data/clean/bep_company_addresses_all.csv", index=False)

#### Geocode Physical Company Addresses

In [13]:
# Create DataFrame of unique addresses
physical_df = address_df.copy().query("`Address Type` == 'Physical'")
physical_df.set_index("Company Name", inplace=True)
physical_df['Street'] = physical_df['Street'].str.strip(".")
address_cols = ['Street', 'City', 'State', 'ZIP Code']
unique_address_df = physical_df[address_cols].drop_duplicates()

# Preview DataFrame
unique_address_df.head()

,Street,City,State,ZIP Code
Company Name,,,,
#1 Professional Cleaning Services,2205 Mayfair,Westchester,IL,60154
"#9 Design, LLC",524 West Saint Charles Road,Villa Park,IL,60181
'Bouche' Enterprises LLC,3511 North Osceola Avenue,Chicago,IL,60634
'Merica Made Painting & Decorating Corp.,6221 S Austin Ave,Chicago,IL,60638
*VDR Development Group,168 N Brandon Drive,Glendale Heights,IL,60190


In [14]:
# Initialize variables
base_url = "http://dev.virtualearth.net/REST/v1/Locations/US/{state}/{zip_code}/{city}/{street_address}"
api_key = os.environ['BING_API_KEY']

# Load cache from disk or create if it doesn't exist
os.makedirs("../data/cache", exist_ok=True)
cache_fpath = "../data/cache/geocoded_addresses.json"
if os.path.exists(cache_fpath):
    with open(cache_fpath) as f:
        cache = json.load(f)
else:
    cache = {}

# Geocode each address
try:
    for idx, row in unique_address_df.iterrows():

        # Log company to be geocoded
        print(f"Geocoding company \"{idx}\".")

        # Skip API call if address has already been geocoded
        if idx in cache:
            print("Company already in cache. Skipping.")
            continue
        
        # Build request URL
        # NOTE: Encoding only used to escape hash symbol
        url = base_url.format(
            state=row['State'],
            zip_code=row['ZIP Code'],
            city=row['City'],
            street_address=row['Street'].strip(',').replace("#", "%23"))
        params = ["verboseplacenames=true", f"key={api_key}"]
        url = f"{url}?{'&'.join(params)}"

        # Attempt to forward geocode address
        r = requests.get(url, timeout=30)

        # Raise exception if geocoding failed
        if not r.ok:
            print(f"The Bing Maps API response returned "
                f"a status code of \"{r.reason} - {r.status_code}\" "
                f"and the text '{r.text}'.")
            continue
            
        # Raise exception if no results returned
        try:
            resource_set = r.json()["resourceSets"][0]
            top_resource = resource_set["resources"][0]
            address = top_resource["address"]
        except (KeyError, IndexError) as e:
            raise Exception("No address results contained in "
                f"Bing Maps API response. {e}")
        
        # Otherwise, parse response for expected properties
        try:
            lat, lon = top_resource['point']['coordinates']
            state = address["adminDistrict"]
            county = address.get("adminDistrict2", "")
            city = address['locality']
            street_address = address.get('addressLine', '')
            confidence = top_resource['confidence']
        except KeyError as e:
            raise Exception("Geocoded response missing "
                f"expected property '{e}'.")

        # Add geocoded address to cache
        cache[idx] = {
            "Latitude": lat,
            "Longitude": lon,
            "Street": street_address,
            "City": city,
            "County": county,
            "State": state,
            "Confidence": confidence
        }

        # Pause before making next request
        time.sleep(1)

except Exception as e:
    # Log error
    print(f"Failed to geocode company addresses. {e}")
    print("Persisting any entries to cache.")
finally:
    with open(cache_fpath, "w") as f:
        json.dump(cache, f, indent=2)

print("Geocoding complete.")
    


Geocoding company "#1 Professional Cleaning Services".
Company already in cache. Skipping.
Geocoding company "#9 Design, LLC".
Company already in cache. Skipping.
Geocoding company "'Bouche' Enterprises LLC".
Company already in cache. Skipping.
Geocoding company "'Merica Made Painting & Decorating Corp.".
Company already in cache. Skipping.
Geocoding company "*VDR Development Group".
Company already in cache. Skipping.
Geocoding company "1052 Partners, Inc. dba PeopleCaddie".
Company already in cache. Skipping.
Geocoding company "123 Labor Inc.".
Company already in cache. Skipping.
Geocoding company "1822 Enterprises LLC".
Company already in cache. Skipping.
Geocoding company "1st Aerow Valuation Group, LLC".
Company already in cache. Skipping.
Geocoding company "1st Call Technical Services, Inc".
Company already in cache. Skipping.
Geocoding company "1st Choice Specialty Services Inc.".
Company already in cache. Skipping.
Geocoding company "1st Metropolitan Translation Services, Inc."

In [15]:
# Create new DataFrame from geocoded addresses
geocoded_df = (pd
    .DataFrame
    .from_dict(cache, orient="index")
    .reset_index()
    .rename(columns={"index": "Company Name"}))

# Merge geocoded addresses to company names
geocoded_companies_df = physical_address_df[['Company Name']].merge(
    right=geocoded_df,
    how="left",
    on="Company Name")

# Preview DataFrame
geocoded_companies_df


,Company Name,Latitude,Longitude,Street,City,County,State,Confidence
0,#1 Professional Cleaning Services,41.848017,-87.891201,2205 Mayfair Ave,Westchester,Cook County,IL,High
1,"#9 Design, LLC",41.890007,-87.992682,524 W Saint Charles Rd,Villa Park,DuPage County,IL,High
2,'Bouche' Enterprises LLC,41.943783,-87.812819,3511 N Osceola Ave,Chicago,Cook County,IL,High
3,'Merica Made Painting & Decorating Corp.,41.779011,-87.771186,6221 S Austin Ave,Chicago,Cook County,IL,High
4,*VDR Development Group,41.931275,-88.083132,168 N Brandon Dr,Glendale Heights,DuPage County,IL,High
...,...,...,...,...,...,...,...,...
4972,mk communications,41.890224,-87.637564,350 W Hubbard St,Chicago,Cook County,IL,High
4973,"myWHY Agency, Inc.",41.777845,-87.601262,6425 S Ingleside Ave,Chicago,Cook County,IL,High
4974,netlogx LLC,39.772103,-86.149574,429 E Vermont St,Indianapolis,Marion County,IN,High
4975,"r6catalyst, LLC",39.872830,-74.811213,617 Stokes Rd,Medford,Burlington County,NJ,High


In [16]:
# Define column for full street address
physical_address_df['Original Address'] = physical_address_df["Street"].apply(lambda s: s.replace(".", "")) + \
    ", " + physical_address_df["City"] + ", " + \
    physical_address_df["State"]

# Merge column to geocoded addresses as "Original Address"
geocoded_companies_df = geocoded_companies_df.merge(
    right=physical_address_df[['Company Name', 'Original Address']],
    how="left",
    on="Company Name")

geocoded_companies_df.head()

,Company Name,Latitude,Longitude,Street,City,County,State,Confidence,Original Address
0,#1 Professional Cleaning Services,41.848017,-87.891201,2205 Mayfair Ave,Westchester,Cook County,IL,High,"2205 Mayfair, Westchester, IL"
1,"#9 Design, LLC",41.890007,-87.992682,524 W Saint Charles Rd,Villa Park,DuPage County,IL,High,"524 West Saint Charles Road, Villa Park, IL"
2,'Bouche' Enterprises LLC,41.943783,-87.812819,3511 N Osceola Ave,Chicago,Cook County,IL,High,"3511 North Osceola Avenue, Chicago, IL"
3,'Merica Made Painting & Decorating Corp.,41.779011,-87.771186,6221 S Austin Ave,Chicago,Cook County,IL,High,"6221 S Austin Ave, Chicago, IL"
4,*VDR Development Group,41.931275,-88.083132,168 N Brandon Dr,Glendale Heights,DuPage County,IL,High,"168 N Brandon Drive, Glendale Heights, IL"


In [17]:
# Get original addresses of companies that may have been skipped during
# the geocoding process because they shared the same address with another entry
failures = geocoded_companies_df.query("Latitude != Latitude")['Original Address'].tolist()
ref_df = geocoded_companies_df.query("`Original Address` in @failures").groupby('Original Address').first().reset_index()

# Define local function to correct skipped companies
def fix_failures(row: pd.Series):
    original_addr = row['Original Address'] 
    if original_addr in failures:
        ref = ref_df.query("`Original Address` == @original_addr").iloc[0]
        return [
            row['Company Name'],
            ref['Latitude'],
            ref['Longitude'],
            ref['Street'],
            ref['City'],
            ref['County'],
            ref['State'],
            ref['Confidence'],
            original_addr
        ]

    else:
        return row

# Apply function across DataFrame
fixed_geocoded = geocoded_companies_df.apply(fix_failures, axis="columns", result_type='broadcast')

# Determine what rows are still not successfully geocoded, if any
# TODO: Address these entries manually or with a Python package
# that performs address normalization
fixed_geocoded.query("Latitude != Latitude")

,Company Name,Latitude,Longitude,Street,City,County,State,Confidence,Original Address
883,"Blue Timber, LLC",NaN,NaN,None,None,None,None,None,"60 W Terra Cotta Ave Ste: B 163, Crystal Lake, IL"
1398,"Combined Card Services, Inc. D/b/a Viva's Hall...",NaN,NaN,None,None,None,None,None,"5757 S Cicero Ave,, Chicago, IL"
1727,"Delivery & Distribution Solutions, LLC",NaN,NaN,None,None,None,None,None,"16W 251 S Frontage Rd, Unit 21 & 22, Burr Ridg..."
2004,EmeraldAI Technologies LLC,NaN,NaN,None,None,None,None,None,"4 Cortland Cir, Attn: Susan Stellflue, Lake Zu..."
2351,"Galaxy Environmental, Inc.",NaN,NaN,None,None,None,None,None,"3565 N Milwaukee Ave, Attn: George Salinas, Ch..."
2682,"HireLevel, Inc.",NaN,NaN,None,None,None,None,None,"3016 Locust St Suite 102 & 103, St. Louis, MO"
3315,"LAW Realty, Ltd.",NaN,NaN,None,None,None,None,None,"2404 S Wolcott Ave, Unit 21 Suite: 1A & B, Chi..."
3316,"LAW Ventures, Ltd.",NaN,NaN,None,None,None,None,None,"2404 S Wolcott Unit 21 Suite 1 A & B, Chicago, IL"
3808,Mega Steel Corp.,NaN,NaN,None,None,None,None,None,"556 County Line Road, Suite D, Attn: Arlene Ba..."
4559,"QU-BAR, INC",NaN,NaN,None,None,None,None,None,"4163 A&B W 166th Street, Oak Forest, IL"


In [18]:
# Write DataFrame to storage
fixed_geocoded.to_csv("../data/clean/bep_company_addresses_physical_geocoded.csv", index=False)